In [8]:
import csv
import os
import pandas as pd
csv.field_size_limit(1000000000)

# read csv file from data folder
df = pd.read_csv(os.path.join('data', 'state-of-the-union.csv'), skiprows=0, names=['year','text'])
# df.columns = ['year', 'text']
# see first 10 rows
print(df.head(1))

   year                                               text
0  1790  George Washington\nJanuary 8, 1790\n\nFellow-C...


In [9]:
print(df['year'])
print(df['text'])


0      1790
1      1790
2      1791
3      1792
4      1793
       ... 
221    2008
222    2009
223    2010
224    2011
225    2012
Name: year, Length: 226, dtype: int64
0      George Washington\nJanuary 8, 1790\n\nFellow-C...
1      \nState of the Union Address\nGeorge Washingto...
2      \nState of the Union Address\nGeorge Washingto...
3      \nState of the Union Address\nGeorge Washingto...
4      \nState of the Union Address\nGeorge Washingto...
                             ...                        
221    \nState of the Union Address\nGeorge W. Bush\n...
222    \nAddress to Joint Session of Congress \nBarac...
223    \nState of the Union Address\nBarack Obama \nJ...
224    \nState of the Union Address\nBarack Obama \nJ...
225    \nState of the Union Address\nBarack Obama \nJ...
Name: text, Length: 226, dtype: object


In [12]:
df[1:1]

,year,text
